# FTX reader

In [1]:
from jupyter_imports import *
from utils.io_utils import *
from histfeed.ftx_history import main

end_time= datetime.now(timezone.utc)-timedelta(days=1)#.replace(minute=0,second=0,microsecond=0)
start_time= end_time-timedelta(days=60)
halflife=timedelta(days=30)
modes=[('h','high'),('c','close'),('l','low'),('v','size')]

In [2]:
weights = pd.read_csv(os.path.join(os.sep, 'home','david',"config", 'prod',"pfoptimizer",'weights_ftx_debug.csv')).sort_values(by='optimalWeight',key=lambda f: np.abs(f),ascending=False)
coins = list(weights.set_index(['name']).index.map(lambda f: f.replace('-PERP','')))
weights

,name,spot,mark,index,borrow,quote_borrow,ExpectedBenchmark,funding,previousWeight,optimalWeight,...,intShortCarry,intBorrow,intUSDborrow,E_long,E_short,E_intBorrow,E_intUSDborrow,direction,exchange,subaccount
0,LINK-PERP,7.295,7.3025,7.295641,0.020079,0.026772,0.000185,0.105192,1.4914,1.000001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
1,AAVE-PERP,94.390,94.3100,94.345029,0.011742,0.026772,-0.000267,-0.087660,-8.8578,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
2,DOT-PERP,8.005,8.0000,8.007660,0.134094,0.026772,-0.000725,-0.394470,0.0000,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
3,AXS-PERP,17.284,17.2830,17.298334,0.536142,0.026772,-0.001873,-0.587322,0.0000,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
4,MKR-PERP,1061.000,1059.5000,1060.898741,0.011742,0.026772,-0.000134,-0.131490,-4.2980,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug


In [3]:
futures_df = main('get','ftx','wide',90)

INFO:histfeed:running histfeed args=('get', 'ftx', 'wide', 90) kwargs={}
INFO:histfeed:Getting history...
INFO:histfeed:command returned                            1INCH-PERP_rate_funding  AAVE-PERP_rate_funding  \
2022-06-09 15:00:00+00:00                 0.087660               -0.140256   
2022-06-09 16:00:00+00:00                 0.026298               -0.035064   
2022-06-09 17:00:00+00:00                 0.078894               -0.078894   
2022-06-09 18:00:00+00:00                 0.078894                0.000000   
2022-06-09 19:00:00+00:00                 0.078894                0.017532   
...                                            ...                     ...   
2022-09-07 11:00:00+00:00                -0.008766               -0.061362   
2022-09-07 12:00:00+00:00                -0.175320               -0.166554   
2022-09-07 13:00:00+00:00                -0.131490               -0.262980   
2022-09-07 14:00:00+00:00                -0.140256               -0.254214   
2022-

### rates markets

In [8]:
futures_df.columns


Index(['1INCH-PERP_rate_funding', 'AAVE-PERP_rate_funding',
       'ATOM-PERP_rate_funding', 'AVAX-PERP_rate_funding',
       'AXS-PERP_rate_funding', 'BCH-PERP_rate_funding',
       'BNB-PERP_rate_funding', 'BTC-PERP_rate_funding',
       'CEL-PERP_rate_funding', 'DOGE-PERP_rate_funding',
       ...
       'TRX_rate_borrow', 'TRX_rate_size', 'UNI_rate_borrow', 'UNI_rate_size',
       'USDT_rate_borrow', 'USDT_rate_size', 'XRP_rate_borrow',
       'XRP_rate_size', 'USD_rate_borrow', 'USD_rate_size'],
      dtype='object', length=577)

In [4]:
coins = ['AVAX']

In [5]:
selected = [df for df in futures_df.columns if any(df.split('-')[0].split('_')[0]==coin for coin in (coins+['USD']))]
prices=futures_df[[c for c in selected if any(string in c for string in ['price_c'])]]

short_rates=futures_df[[c for c in selected if any(string in c for string in ['_rate_c','rate_borrow'])]]
short_rates=short_rates.ewm(halflife=halflife,times=short_rates.index).mean()

borrow_size=futures_df[[c for c in selected if any(string in c for string in ['_rate_size','price_volume'])]]

for c in coins:
    long_perp_carry=short_rates[c+'-PERP_rate_c']-short_rates['USD_rate_borrow']
    short_perp_carry=-short_rates[c+'-PERP_rate_c']-short_rates[c+'_rate_borrow']
    short_rates[c+'_best_perp_carry']= pd.concat([long_perp_carry, short_perp_carry]).groupby(level=0).max()
    # long_rates[c+'_long_future_carry']=long_rates[c+'-1231_rate_c']-short_rates['USD_rate_borrow']
    # long_rates[c+'_short_future_carry']=-long_rates[c+'-1231_rate_c']-short_rates[c+'_rate_borrow']

short_rates.iplot()#))

In [6]:
futures_df.filter(like='AVAX-PERP_rate_c').ewm(halflife=halflife,times=short_rates.index).mean().iplot()

In [7]:
a =pd.DataFrame()
halflife=timedelta(days=30)
a['fut'] = futures_df.filter(like='AVAX-0930_rate_c')
a['perp']=futures_df.filter(like='AVAX-PERP_rate_c').ewm(halflife=halflife,times=a.index).mean()
a.iplot()

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
pd.concat([borrow_size],axis=0).iplot()#))

### carry directionality

In [ ]:
plotthat = futures_df[['BNB-PERP_rate_h',
       'BNB-PERP_rate_l', 'BNB-PERP_rate_c']]
plotthat.columns=['h','l','c']

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plotthat.index,#+x_rev,
    y=0.5*(plotthat['l'].values+plotthat['h'].values),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='BNB',
))
fig.add_trace(go.Scatter(
    x=plotthat.index, y=plotthat['c'].values,
    line_color='rgb(0,100,80)',
    name='Fair',
))

fig.update_traces(mode='lines')
fig.show()

In [ ]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [ ]:
end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(hours=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compute_premium_vs_spot(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/price/'+modes[mode][0]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
granular_df['premium']=compute_premium_vs_spot(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]
granular_df[['premium']].iplot()

In [ ]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [ ]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))